In [1]:
import os
import torch
from use_model import load_fine_tuned_model, generate_response

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

torch.cuda.empty_cache()
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.get_device_name()
torch.cuda.memory_allocated()
torch.cuda.memory_reserved()
torch.cuda.memory_summary()

# Load the model once
model, tokenizer = load_fine_tuned_model()

# Use it multiple times
response = generate_response(
    model,
    tokenizer,
    question="What is this about?",
    context="Your document text here"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


OutOfMemoryError: CUDA out of memory. Tried to allocate 492.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 363.44 MiB is free. Including non-PyTorch memory, this process has 4.95 GiB memory in use. Of the allocated memory 4.85 GiB is allocated by PyTorch, and 19.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

.